# Meta-Informationen

In den letzten Teilen haben wir uns intensiv mit den Informationen der Texte selbst beschäftigt und du konntest daraus schon einige Erkenntnisse ableiten.

Häufig stehen in den Dokumenten auch strukturierte Informationen zur Verfügung. Im Fall des Heise Newstickers sind das Zeitinformationen, Autoren und Keywords. 

## Daten auswählen

Wie üblich beginnst du zunächst mit der Datenauswahl. Weil wir jetzt auch Zeitstempel analysieren möchten, solltest du darauf achten, dass sich keine zu neuen Artikel *eingeschlichen haben*:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM articles \
                   WHERE datePublished<'2021-01-01' \
                   ORDER BY datePublished", sql, index_col="id", parse_dates=["datePublished"])

## Zeitverlauf 

Zunächst interessieren wir uns für den zeitlichen Verlauf der Artikel. Weil du dich nicht für die Zeiten selbst interessierst, sondern für die Anzahl der Artikel pro Tag, nutzt du die `pandas`-Funktion `resample`. Als Aggregation verwendest du `count()`, weil die die Artikel zählen möchtest. Ein Feld, also z.B. die `url`, genügt zur Analyse:

In [ ]:
df.set_index("datePublished").resample('D').count()[["url"]].plot()

Das hat nicht geklappt! In einem der früheren Schritte hast du `datePublished` aus dem `json+ld` übernommen, dabei stand offenbar manchmal Sommerzeit oder Winterzeit als Zeitstempel drin.

`pandas` kann das für dich in [UTC](https://de.wikipedia.org/wiki/Koordinierte_Weltzeit) umrechnen:

In [ ]:
df["datePublished"] = pd.to_datetime(df["datePublished"], utc=True)

Nun sollte es klappen:

In [ ]:
df.set_index("datePublished").resample('D').count()[["url"]].plot()

Sehr viel kannst du noch nicht erkennen, hauptsächlich die Wochenenden, an denen weniger Artikel veröffentlicht wurden. `resample` kann auch nach Wochen aggregieren:

In [ ]:
df.set_index("datePublished").resample('W').count()[["url"]].plot()

Eine Woche geht noch mit in das Jahr 2021. Wenn du möchtest, kannst du es auch noch nach Monaten aggregieren:

In [ ]:
df.set_index("datePublished").resample('M')[["url"]].count().plot()

Auch die Heise-Redakteure sind im August offenbar im Urlaub 😊.

## Autoren

Mithilfe der `pandas`-Funktion `groupby` kannst du überprüfen, welcher Autor am meisten schreibt:

In [ ]:
top_authors = df.groupby("author").count()[["url"]].sort_values("url")[-20:]
top_authors.plot.barh()

Spannend könnte auch sein, welcher Autor in welchem Monat am meisten schreibt. Dazu definierst du dir zunächst eine Spalte `month`, die nur den Monat der Publikation enthält:

In [ ]:
df["month"] = df["datePublished"].dt.strftime("%m")

`pandas` kann auch nach mehreren Spalten gruppieren, das machst du nun für `author` und `month`. Dabei zählst du die Anzahl der Artikel und behältst nur die Kombinationen aus Autor und Monat, die größer als 10 sind:

In [ ]:
am = df.groupby(["author", "month"], as_index=False).agg({"url": "count"})
am = am[am["url"]>10]
am

Die Tabelle muss nun *aufgeklappt* werden, du möchtest `month` gerne als Spalten haben. Dafür dient die `pandas`-Funktion `pivot`. Da es Kombinationen gibt, die nicht belegt sind, füllst du diese mit 0 (`fillna(0)`) und zeichnest das Ergebnis mittels `seaborn` als Heatmap:

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
sns.heatmap(am.pivot(index="author", columns="month", values="url").fillna(0))

Das ist schon sehr interessant und etwas indiskret, weil du den Urlaub der Heise-Redakteure ablesen kannst.

## Autoren vs. Keywords

Evtl. interessiert dich nun, welcher Autor zu welchem Thema schreibt. Allerdings kann `pandas` nicht gut mit mehrwertigen Feldern wie `keywords` umgehen. Deswegen definierst du dir einen neuen `DataFrame`, der für jeden `author` und jedes `keyword` als `count` den Wert 1 enthält:

In [ ]:
ak = pd.DataFrame([[author, k, 1] for (author, keywords) in zip(df["author"], df["keywords"]) 
                         for k in str(keywords).split(", ")], columns=["author", "keyword", "count"]).dropna()

Nun zählst du, wie häufig die Kombination aus `author` und `keyword` vorkommt:

In [ ]:
akg = ak.groupby(["author", "keyword"], as_index=False).count()

Damit es nicht zu unübersichtlich wird, nutzt du nur Kombinationen, die mindestens 20mal auftreten:

In [ ]:
akg_top = akg[akg["count"]>50].pivot(index="author", columns="keyword", values="count").fillna(0)

Das Ergebnis kannst du wieder als Heatmap darstellen und hast damit eine Art *Wissenslandkarte* der Heise-Redakteure erstellt:

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(akg_top)

## Metadaten sind wichtig!

Obwohl wir uns im Kurs hautpsächlich mit Textanalyse beschäftigen, solltest du die Metadaten nicht vergessen. Sie geben die wichtige Zusatzinformationen und außerdem ein gutes Gefühl dafür, wie hochwertig die Daten sind.